In [15]:
#import 20newsgroups datasets from sklearn
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',remove=(['headers','footers', 'quotes']))

In [16]:
#list of categories(labels)
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
#counts the occurence of each word. 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape
# count_vect.vocabulary_.get(u'algorithm')

(11314, 101631)

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
#transform from occurrences to tf-idf 
#tf-idf: 1. Decide the number of occurrences of each word in a document by total number of words in document. (Term Frequences tf)
#        2. It does another refinement. Downscaling weights for words that occurring in many documents.
tfidf_transformer = TfidfTransformer()

#use X_train_tfidf to train the MODEL
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#none tfidf.
X_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
#use docs_test to fit.
 

In [19]:
 #   Training a classifier example:
 #   from sklearn.naive_bayes import MultinomialNB
 #   clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
    


In [14]:
#  Predict an outcome on new document:
#  Doc_examples = ['God is love', 'GPU is fast']

#  convert documents into tf-idf.  
#  X_new_counts = count_vect.transform(Doc_examples) 
#  X_new_tfidf = tfidf_transformer.transform(X_new_counts)


#  predicted = clf.predict(X_new_tfidf)   (name_of_your_model_object)
#  it stores the prediction of doc_examples.


